<a href="https://colab.research.google.com/github/ShivangiModi/Retail-Product-Self-Checkout-System/blob/main/Retail_Database_Webpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask opencv-python torch torchvision pillow pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok config add-authtoken 2ueK7mAO9lTjjkw5IwrM2YSGs9T_35QUnzsg2QTCgQTGzLutS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask flask-ngrok

In [ ]:
!pip install flask flask-ngrok
from flask import Flask, request, jsonify
from pyngrok import ngrok
import cv2
import numpy as np
import pandas as pd
import torch
import subprocess

# Load trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Reduce_new_data/split/yolov5/runs/train/exp3/weights/best.pt', force_reload=True)

# Load product price list
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')  # Ensure this file exists with columns: "Product", "Price"

# Flask API
app = Flask(__name__)

@app.route("/get_ngrok_url")
def get_ngrok_url():
    try:
        # Run `ngrok` command to get the public URL dynamically
        result = subprocess.run(["curl", "-s", "http://localhost:5000/api/tunnels"], capture_output=True, text=True)
        if result.returncode == 0:
            import json
            tunnels = json.loads(result.stdout)["tunnels"]
            public_url = tunnels[0]["public_url"] if tunnels else None
            return jsonify({"ngrok_url": public_url})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    return jsonify({"ngrok_url": None})

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/fronend_retail.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    # Receive image from frontend
    file = request.files['image']
    image_path = "/content/drive/MyDrive/Reduce_new_data/test2019/20180824-14-05-08-13.jpg"
    file.save(image_path)

    # Load image and run YOLO detection
    img = cv2.imread(image_path)
    results = model(img)
    print(results)

    # Extract detected product names
    detected_products = results.pandas().xyxy[0]['name'].tolist()

    # Generate bill
    bill = []
    total_price = 0
    for product in detected_products:
        if product in billing_data['Product'].values:
            price = billing_data[billing_data['Product'] == product]['Price'].values[0]
            bill.append({'Product': product, 'Price': price})
            total_price += price

    return jsonify({'detected_products': detected_products, 'bill': bill, 'total_price': total_price})

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
print("hello")
print(app.url_map)
# Run Flask server
app.run(port=5000)


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-4-1 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


Public URL: NgrokTunnel: "https://20fb-35-229-109-29.ngrok-free.app" -> "http://localhost:5000"
hello
Map([<Rule '/static/<filename>' (HEAD, OPTIONS, GET) -> static>,
 <Rule '/get_ngrok_url' (HEAD, OPTIONS, GET) -> get_ngrok_url>,
 <Rule '/' (HEAD, OPTIONS, GET) -> home>,
 <Rule '/upload' (OPTIONS, POST) -> upload_image>])
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
          

In [ ]:
!pip install ultralytics

from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.2/984.2 kB 17.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv8 trained model - 150 epochs - 1000 and 500
model = YOLO('/content/drive/MyDrive/Val/runs/detect/val_split_1000/weights/best.pt')


# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']  # Ensure correct column names

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)

    # Run YOLOv5 on the uploaded image
    img = cv2.imread(image_path)
    results = model(img)

    # Extract detected product names
    detected_products = []
    for result in results:
        class_indices = result.boxes.cls.tolist()  # Extract class indices
        names_dict = result.names  # Class ID to Name mapping
        detected_products.extend([names_dict[int(idx)] for idx in class_indices])

    # Generate the bill
    bill = []
    total_price = 0
    for product in detected_products:
        product_info = billing_data[billing_data['Product'] == product]
        if not product_info.empty:
            price = product_info['Price'].values[0]
            bill.append({'Product': product, 'Price': float(price)})
            total_price += float(price)

    response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price}
    print("Response:", response)  # Debugging log
    return jsonify({'detected_products': detected_products, 'bill': bill, 'total_price': total_price})


if __name__ == '__main__':
    print("Server is starting...")
    app.run()


Public URL: NgrokTunnel: "https://bd2a-34-148-111-178.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bd2a-34-148-111-178.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 20:09:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 20:09:52] "GET /favicon.ico HTTP/1.1" 404 -



0: 640x640 3 Drinks, 3 Chocolates, 3 Tissues, 1268.6ms
Speed: 25.4ms preprocess, 1268.6ms inference, 41.8ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 20:11:05] "POST /upload HTTP/1.1" 200 -


Response: {'detected_products': ['Drink', 'Tissue', 'Tissue', 'Drink', 'Drink', 'Chocolate', 'Chocolate', 'Tissue', 'Chocolate'], 'bill': [{'Product': 'Drink', 'Price': 1.99}, {'Product': 'Tissue', 'Price': 2.49}, {'Product': 'Tissue', 'Price': 2.49}, {'Product': 'Drink', 'Price': 1.99}, {'Product': 'Drink', 'Price': 1.99}, {'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Tissue', 'Price': 2.49}, {'Product': 'Chocolate', 'Price': 2.49}], 'total_price': 20.910000000000004}


INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 20:11:41] "GET /payment?amount=20.91 HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv8 trained model - 1800/600 images - 180 epochs
model = YOLO('/content/drive/MyDrive/retail_product_checkout_validation_filtered/runs/detect/train/weights/best.pt')

# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']  # Ensure correct column names

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)

    # Run YOLOv5 on the uploaded image
    img = cv2.imread(image_path)
    results = model(img)

    # Extract detected product names
    detected_products = []
    for result in results:
        class_indices = result.boxes.cls.tolist()  # Extract class indices
        names_dict = result.names  # Class ID to Name mapping
        detected_products.extend([names_dict[int(idx)] for idx in class_indices])

    # Generate the bill
    bill = []
    total_price = 0
    for product in detected_products:
        product_info = billing_data[billing_data['Product'] == product]
        if not product_info.empty:
            price = product_info['Price'].values[0]
            bill.append({'Product': product, 'Price': float(price)})
            total_price += float(price)

    response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price}
    print("Response:", response)  # Debugging log
    return jsonify({'detected_products': detected_products, 'bill': bill, 'total_price': total_price})


if __name__ == '__main__':
    print("Server is starting...")
    app.run()

Public URL: NgrokTunnel: "https://b6f4-34-48-116-230.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b6f4-34-48-116-230.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 18:19:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 18:19:52] "GET /favicon.ico HTTP/1.1" 404 -



0: 640x640 3 Chocolates, 3 Drinks, 3 Puffed_Foods, 221.4ms
Speed: 10.1ms preprocess, 221.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 18:19:59] "POST /upload HTTP/1.1" 200 -


Response: {'detected_products': ['Chocolate', 'Puffed_Food', 'Puffed_Food', 'Chocolate', 'Chocolate', 'Puffed_Food', 'Drink', 'Drink', 'Drink'], 'bill': [{'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Puffed_Food', 'Price': 2.79}, {'Product': 'Puffed_Food', 'Price': 2.79}, {'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Puffed_Food', 'Price': 2.79}, {'Product': 'Drink', 'Price': 1.99}, {'Product': 'Drink', 'Price': 1.99}, {'Product': 'Drink', 'Price': 1.99}], 'total_price': 21.809999999999995}


In [ ]:
# Final Yolov5, Yolov8 Model with Payment

In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os
from flask import send_from_directory

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv5 trained model - shivangi
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/Reduce_new_data/split/yolov5/runs/train/exp5/weights/best.pt', force_reload=True)

# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']  # Ensure correct column names

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage_design.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)

    # Read the image
    img = cv2.imread(image_path)

    # Run YOLOv5 detection
    results = model(img)

    # Get detected class indices
    results_df = results.pandas().xyxy[0]
    class_indices = results_df['name'].tolist()  # name is the product class
    detected_products = class_indices  # Already strings

    #detected_products = [names_dict[int(idx)] for idx in class_indices]

    # Generate the bill
    bill = []
    total_price = 0
    for product in detected_products:
        product_info = billing_data[billing_data['Product'] == product]
        if not product_info.empty:
            price = product_info['Price'].values[0]
            bill.append({'Product': product, 'Price': float(price)})
            total_price += float(price)

    response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price}
    print("Response:", response)  # Debugging
    return jsonify(response)

@app.route('/payment')
def payment_page():
    amount = request.args.get('amount', '0.00')
    return send_file('/content/drive/MyDrive/Reduce_new_data/payment.html')


@app.route('/payment/success', methods=['GET'])
def payment_success():
    return send_file('/content/drive/MyDrive/Reduce_new_data/success.html')

if __name__ == '__main__':
    print("Server is starting...")
    app.run()


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-4-25 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


Public URL: NgrokTunnel: "https://834b-34-48-116-230.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://834b-34-48-116-230.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:04] "GET /favicon.ico HTTP/1.1" 404 -
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:11] "POST /upload HTTP/1.1" 200 -


Response: {'detected_products': ['Chocolate', 'Dessert', 'Dried_Fruit', 'Puffed_Food', 'Dried_Fruit', 'Dessert', 'Dessert', 'Chocolate', 'Instant_Drink', 'Instant_Drink', 'Tissue', 'Dried_Fruit', 'Personal_Hygiene'], 'bill': [{'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Dried_Fruit', 'Price': 6.99}, {'Product': 'Puffed_Food', 'Price': 2.79}, {'Product': 'Dried_Fruit', 'Price': 6.99}, {'Product': 'Chocolate', 'Price': 2.49}, {'Product': 'Instant_Drink', 'Price': 1.79}, {'Product': 'Instant_Drink', 'Price': 1.79}, {'Product': 'Tissue', 'Price': 2.49}, {'Product': 'Dried_Fruit', 'Price': 6.99}, {'Product': 'Personal_Hygiene', 'Price': 3.49}], 'total_price': 38.300000000000004}


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:12] "GET /payment?amount=38.30 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:17] "GET /payment/success HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:48:20] "GET / HTTP/1.1" 304 -


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os
from flask import send_from_directory

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv8 trained model - Vatsal
model = YOLO('/content/drive/MyDrive/Val/runs/detect/val_split_1000/weights/best.pt')

# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage_design.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    try:
        file = request.files['image']
        image_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(image_path)

        img = cv2.imread(image_path)

        # Run YOLOv8 detection
        results = model(img)
        result = results[0]

        # Get class indices and map to names
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        detected_products = [model.names[i] for i in class_ids]

        # Generate bill
        bill = []
        total_price = 0
        for product in detected_products:
            product_info = billing_data[billing_data['Product'] == product]
            if not product_info.empty:
                price = product_info['Price'].values[0]
                bill.append({'Product': product, 'Price': float(price)})
                total_price += float(price)

        response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price}
        return jsonify(response)

    except Exception as e:
        print("Error during upload_image:", str(e))
        return jsonify({'error': str(e)}), 500


@app.route('/payment')
def payment_page():
    amount = request.args.get('amount', '0.00')
    return send_file('/content/drive/MyDrive/Reduce_new_data/payment.html')


@app.route('/payment/success', methods=['GET'])
def payment_success():
    return send_file('/content/drive/MyDrive/Reduce_new_data/success.html')

if __name__ == '__main__':
    print("Server is starting...")
    app.run()


Public URL: NgrokTunnel: "https://b62b-34-48-116-230.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b62b-34-48-116-230.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:02] "GET /favicon.ico HTTP/1.1" 404 -



0: 640x640 3 Drinks, 3 Chocolates, 3 Tissues, 625.0ms
Speed: 10.2ms preprocess, 625.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:08] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:13] "GET /payment?amount=20.91 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:16] "GET /payment/success HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 17:54:17] "GET / HTTP/1.1" 304 -


In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os
from flask import send_from_directory
from collections import Counter
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from io import BytesIO

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv8 trained model - Vaishnavi - 180 epoch
model = YOLO('/content/drive/MyDrive/retail_product_checkout_validation_filtered/runs/detect/train/weights/best.pt')

# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage_design.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    try:
        file = request.files['image']
        image_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(image_path)

        img = cv2.imread(image_path)

        # Run YOLOv8 detection
        results = model(img)
        result = results[0]

        # Get class indices and map to names
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        detected_products = [model.names[i] for i in class_ids]

        # Generate bill
        bill = []
        total_price = 0

        # Count occurrences of each product
        product_counts = Counter(detected_products)

        for product, count in product_counts.items():
            product_info = billing_data[billing_data['Product'] == product]
            if not product_info.empty:
                unit_price = float(product_info['Price'].values[0])
                total = unit_price * count
                bill.append({
                    'Product': product,
                    'Quantity': count,
                    'Unit_Price': unit_price,
                    'Total': total
                })
                total_price += total

        response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price, 'total_items': sum(product_counts.values())}
        return jsonify(response)

    except Exception as e:
        print("Error during upload_image:", str(e))
        return jsonify({'error': str(e)}), 500


@app.route('/payment')
def payment_page():
    amount = request.args.get('amount', '0.00')
    return send_file('/content/drive/MyDrive/Reduce_new_data/payment.html')


@app.route('/payment/success', methods=['GET'])
def payment_success():
    return send_file('/content/drive/MyDrive/Reduce_new_data/success.html')

@app.route('/download_bill', methods=['POST'])
def download_bill():
    data = request.get_json()

    bill = data.get('bill', [])
    total_price = data.get('total_price', 0)
    total_items = data.get('total_items', 0)

    # Generate PDF
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter

    y = height - 50
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, "Smart Retail Checkout - Final Bill")
    y -= 40

    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, "Product")
    c.drawString(200, y, "Qty")
    c.drawString(250, y, "Unit Price")
    c.drawString(350, y, "Total")
    y -= 20

    c.setFont("Helvetica", 12)
    for item in bill:
        c.drawString(50, y, item['Product'])
        c.drawString(200, y, str(item['Quantity']))
        c.drawString(250, y, f"${item['Unit_Price']:.2f}")
        c.drawString(350, y, f"${item['Total']:.2f}")
        y -= 20

    y -= 10
    c.line(50, y, 500, y)
    y -= 30
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, f"Total Items: {total_items}")
    c.drawString(250, y, f"Total Price: ${total_price:.2f}")

    c.save()
    buffer.seek(0)

    return send_file(buffer, as_attachment=True, download_name="final_bill.pdf", mimetype='application/pdf')

if __name__ == '__main__':
    print("Server is starting...")
    app.run()


Public URL: NgrokTunnel: "https://3f4b-35-196-253-161.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3f4b-35-196-253-161.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 16:31:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Apr/2025 16:31:52] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify, send_file
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
import pandas as pd
import torch
from pyngrok import ngrok
import os
from flask import send_from_directory
from collections import Counter
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from io import BytesIO

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Automatically starts ngrok

# Load YOLOv8 trained model - Vaishnavi - 180 epoch
model = YOLO('/content/drive/MyDrive/retail_product_checkout_validation_filtered/detect_250_epochs/train/weights/best.pt')

# Load billing data
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')
billing_data.columns = ['Product', 'Price']

# Start ngrok
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Ensure upload directory exists
UPLOAD_FOLDER = "/content/uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def home():
    return send_file('/content/drive/MyDrive/Reduce_new_data/webpage_design.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    try:
        file = request.files['image']
        image_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(image_path)

        img = cv2.imread(image_path)

        # Run YOLOv8 detection
        results = model(img)
        result = results[0]

        # Get class indices and map to names
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        detected_products = [model.names[i] for i in class_ids]

        # Generate bill
        bill = []
        total_price = 0

        # Count occurrences of each product
        product_counts = Counter(detected_products)

        for product, count in product_counts.items():
            product_info = billing_data[billing_data['Product'] == product]
            if not product_info.empty:
                unit_price = float(product_info['Price'].values[0])
                total = unit_price * count
                bill.append({
                    'Product': product,
                    'Quantity': count,
                    'Unit_Price': unit_price,
                    'Total': total
                })
                total_price += total

        response = {'detected_products': detected_products, 'bill': bill, 'total_price': total_price, 'total_items': sum(product_counts.values())}
        return jsonify(response)

    except Exception as e:
        print("Error during upload_image:", str(e))
        return jsonify({'error': str(e)}), 500


@app.route('/payment')
def payment_page():
    amount = request.args.get('amount', '0.00')
    return send_file('/content/drive/MyDrive/Reduce_new_data/payment.html')


@app.route('/payment/success', methods=['GET'])
def payment_success():
    return send_file('/content/drive/MyDrive/Reduce_new_data/success.html')

@app.route('/download_bill', methods=['POST'])
def download_bill():
    data = request.get_json()

    bill = data.get('bill', [])
    total_price = data.get('total_price', 0)
    total_items = data.get('total_items', 0)

    # Generate PDF
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter

    y = height - 50
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, "Smart Retail Checkout - Final Bill")
    y -= 40

    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, "Product")
    c.drawString(200, y, "Qty")
    c.drawString(250, y, "Unit Price")
    c.drawString(350, y, "Total")
    y -= 20

    c.setFont("Helvetica", 12)
    for item in bill:
        c.drawString(50, y, item['Product'])
        c.drawString(200, y, str(item['Quantity']))
        c.drawString(250, y, f"${item['Unit_Price']:.2f}")
        c.drawString(350, y, f"${item['Total']:.2f}")
        y -= 20

    y -= 10
    c.line(50, y, 500, y)
    y -= 30
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, f"Total Items: {total_items}")
    c.drawString(250, y, f"Total Price: ${total_price:.2f}")

    c.save()
    buffer.seek(0)

    return send_file(buffer, as_attachment=True, download_name="final_bill.pdf", mimetype='application/pdf')

if __name__ == '__main__':
    print("Server is starting...")
    app.run()


Public URL: NgrokTunnel: "https://f337-34-48-116-230.ngrok-free.app" -> "http://localhost:5000"
Server is starting...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f337-34-48-116-230.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 20:30:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 20:30:06] "GET /favicon.ico HTTP/1.1" 404 -



0: 640x640 1 Personal_Hygiene, 1 Alcohol, 3 Drinks, 3 Desserts, 237.5ms
Speed: 7.9ms preprocess, 237.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


...

INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 20:31:24] "POST /upload HTTP/1.1" 200 -



0: 640x640 2 Chocolates, 1 Alcohol, 1 Dried_Fruit, 2 Dried_Foods, 221.0ms
Speed: 6.0ms preprocess, 221.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 20:32:00] "POST /upload HTTP/1.1" 200 -



0: 480x640 1 Personal_Hygiene, 1 Puffed_Food, 2 Dried_Foods, 176.5ms
Speed: 5.8ms preprocess, 176.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)


INFO:werkzeug:127.0.0.1 - - [25/Apr/2025 20:33:04] "POST /upload HTTP/1.1" 200 -


In [ ]:
billing_data = pd.read_csv('/content/drive/MyDrive/Reduce_new_data/product_prices.csv')

In [ ]:
billing_data.head()

,Product,Price
0,Canned_Food,2.99
1,Milk,1.49
2,Drink,1.99
3,Alcohol\t,10.99
4,Personal_Hygiene,3.49


In [ ]:
!ngrok --version

ngrok version 3.22.0
pyngrok version 7.2.3
